# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

In [1]:
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pd.set_option("display.max_rows", 1000)

## デレステセリフのスクレイピング

In [3]:
def get_quotes_sute(q):
    """
    starlight.kirara.ca からデレステにおける，
    各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [9]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    icws.indego-bell.com からモバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != n:
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [ ]:
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262
aa = get_quotes_sute(262)

In [15]:
bb = get_quotes_moba(query="白菊ほたる", debug=True)

crawling page: 1/2
crawling page: 2/2


In [13]:
aa.head()

,0
4,アイドルマスター シンデレラガールズ スターライトステージ!
44,ずいぶんと遅い目覚めでしてー、今日のよき日は始まっていますよー
46,お昼でしてー。食はおのれを形作る大切な力ですよー
48,夜遅くまで精が出ますねー、わたくしもご一緒しましょうかー
50,光に満ちているのでしてーLIVEに行くのが吉でしょうー


In [19]:
bb.head()

,0,1,2,3
0,白菊ほたる,白菊ほたる,アイドルコメント,は、はじめまして…白菊ほたるです。実は暗い話で申し訳ないのですが以前所属していたプロダクショ...
1,白菊ほたる,白菊ほたる,マイスタジオ,これで移籍は…何回目か…
2,白菊ほたる,白菊ほたる,マイスタジオ,私がいると…みんな不幸に…
3,白菊ほたる,白菊ほたる,マイスタジオ,すみません…すみません…!
4,白菊ほたる,白菊ほたる,マイスタジオ,あ…○○プロデューサーさん…足元に水たまりが……あっ


## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)